<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/notebooks/dsba6211_summer2024_ProjectMilestone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Milestone 2

DSBA 6211 - Summer 2024

Neal Logan

# Introduction

### Instructions

Introduction and Project Outline:

State your project objective and the chosen dataset.
Describe the specific problem you aim to solve or the question you intend to answer.
Consider and discuss the business application of your project. How could the results of your analysis or model be used in a business context? What potential impact could it have?

### Problem, Application, and Impact

I intend to produce a set of models which predict whether a particular URL is intended to be used for phishing or not.  

Depending on context, this could be used to provide suspected-phishing warnings to users, to block users from visiting these sites, to identify targets for takedown requests, or to otherwise direct the efforts of law enforcement or security or trust & safety teams.  

In general, the impact would be to mitigate security threats from phishing; the mechanisms would vary with use case.

### Dataset

I will use a dataset provided by HuggingFace user pirocheto and hosted on HuggingFace:

https://huggingface.co/datasets/pirocheto/phishing-url


# Literature Review

Find at least two online resources such as blog posts or tutorials that provide a code-implemented project related to your topic. Summarize the key points and explain how they are relevant to your work. Note that the resources should generally not be academic articles unless they include code.[link text](https://)

I asked ChatGPT to find me resources matching these requirements:

### Pirocheto's Phishing URL Detection
https://github.com/pirocheto/phishing-url-detection

This repository contains a complete project for phishing URL detection using machine learning and MLOps practices. It uses a TF-IDF vectorizer using both character and word n-grams) with a linear SVM model. The code is designed to be lightweight and fast, suitable for embedding in applications, and can work offline, without an internet connection. The repository also includes instructions for reproducing the model and running the pipeline.

### PhishShield
https://github.com/praneeth-katuri/PhishShield

This GitHub repository provides a comprehensive solution for detecting phishing websites using advanced machine learning techniques. It includes feature-based and text-based models, both built with scikit-learn. The repository also offers Flask deployment for real-time URL prediction and caching for performance improvement.

### Phishing Link Detection by Sayan Maity
https://github.com/Sayan-Maity-Code/Phishing-link-detection

This project uses Multinomial Naive Bayes and Logistic Regression to detect phishing URLs. The model leverages TF-IDF vectorization and includes scripts for training and evaluating the model. The repository provides a detailed README with instructions for usage and installation.

###Phish-detect by 5h4rk-lab
https://github.com/5h4rk-lab/Phish-detect

This repository provides a tool for detecting phishing URLs which uses RandomForestClassifier from scikit-learn. The tool is built with Python and Flask, making it user-friendly and easy to deploy. The repository includes the necessary code and dependencies to run the project locally.

# Data Preparation

Detail the steps taken to clean and preprocess your data.
Include any transformations, feature engineering, or handling of missing values.

# Methodology

Outline the methods and models you plan to use.
This should include:
For predictive modeling: Describe the models you will compare (e.g., logistic regression, Boosting trees, etc.).
For text mining: Describe the techniques and tools you will use for text processing and analysis (e.g., TF-IDF, etc.).
For forecasting: Describe the time series models you will explore (e.g., ARIMA).

# Preliminary Analysis

Present exploratory data analysis (EDA) to understand the dataset's structure, relationships, and key features.
Include visualizations and summary statistics that support your preliminary findings.


# Initial Results

Conduct initial tests of your models or analysis techniques.
Provide a summary of the results, including any initial performance metrics or insights gained.
Discuss any challenges encountered and how you addressed them.

# Next Steps

Outline the steps you plan to take to refine your approach and improve your results.
Highlight any additional data you might need, further analyses, or alternative methodologies you are considering.